In [1]:
import sys
import os
import platform
import shutil
import logging
from datetime import datetime

os_name = platform.system()
if os_name == "Windows":
    sys.path.insert(0, r"D:/git/pyaedt_library/src/")
else :
    # 두 가지 가능한 경로 패턴
    possible_paths = [
        "/gpfs/home1/r1jae262/jupyter/git/pyaedt_library/src/",
        "/gpfs/home2/wjddn5916/Ansys_NEC/git/pyaedt_library/src/"
    ]
    
    # 존재하는 경로만 sys.path에 추가
    for path in possible_paths:
        if os.path.exists(path):
            sys.path.insert(0, path)

import pyaedt_module
from pyaedt_module.core import pyDesktop

from module.input_parameter import create_input_parameter, set_design_variables


import pandas as pd


class Simulation() :

    def __init__(self, parameter=None):

        self.NUM_CORE = 4
        self.NUM_TASK = 1
        self.parameter = parameter

        # simulation number
        file_path = "simulation_num.txt"
        self.num, self.PROJECT_NAME = get_and_update_simulation_num(file_path)

        # desktop instance
        self.desktop = pyDesktop(version=None, non_graphical=(os_name != "Windows"))


    def create_design(self, name) :
        self.project = self.desktop.create_project()
        self.maxwell_design = self.project.create_design(name=name, solver="Maxwell3d", solution=None)
        return self.maxwell_design


    def create_input_parameter(self) :
        input_parameter = create_input_parameter(self.maxwell_design, self.parameter)
        logging.info(f"input_parameter : {input_parameter}")
        logging.info("input_parameter : " + ",".join(str(float(v)) for v in input_parameter.values()))
        return input_parameter


    def set_variable(self, input_parameter) :

        for key, value in input_parameter.items():
            setattr(self.maxwell_design, key, value)
        
        self.input_df = pd.DataFrame([input_parameter])
        
        set_design_variables(self.maxwell_design, input_parameter)


    def set_maxwell_analysis(self, design, max_passes=12, min_passes=1, min_converged_passes=3, percent_error=2.5, frequency=1) :
        design.setup = design.create_setup(name = "Setup1")
        design.setup.properties["Max. Number of Passes"] = max_passes
        design.setup.properties["Min. Number of Passes"] = min_passes
        design.setup.properties["Min. Converged Passes"] = min_converged_passes
        design.setup.properties["Percent Error"] = percent_error
        design.setup.properties["Frequency Setup"] = f"{frequency}kHz"






def get_and_update_simulation_num(path):
    # initial simulation number
    if not os.path.exists(path):
        with open(path, "w", encoding="utf-8") as f:
            f.write("1")
    # Windows OS case
    if os_name == "Windows":
        with open(path, "r+", encoding="utf-8") as f:
            content = f.read().strip()
            num = int(content) if content else 1
            project_name = f"simulation{num}"
            f.seek(0)
            f.truncate()
            f.write(str(num + 1))
            return num, project_name
    # Linux OS case (fcntl)
    else:
        import fcntl
        with open(path, "r+", encoding="utf-8") as f:
            try:
                fcntl.flock(f.fileno(), fcntl.LOCK_EX)
                content = f.read().strip()
                num = int(content) if content else 1
                project_name = f"simulation{num}"
                f.seek(0)
                f.truncate()
                f.write(str(num + 1))
                return num, project_name
            finally:
                fcntl.flock(f.fileno(), fcntl.LOCK_UN)





simulation_runner = Simulation()

simulation_runner.create_design("MFT_1MW")

input_parameters = simulation_runner.create_input_parameter()

simulation_runner.set_variable(input_parameters)

simulation_runner.set_maxwell_analysis(simulation_runner.maxwell_design, max_passes=3, min_passes=1, min_converged_passes=1, percent_error=2.5, frequency=1)

PyAEDT INFO: Python version 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.23.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\peets\AppData\Local\Temp\pyaedt_peets_5b28da62-aa89-44ec-87f9-ab7586c32681.log is enabled.
PyAEDT INFO: Log on AEDT is disabled.
PyAEDT INFO: Starting new AEDT gRPC session.
PyAEDT INFO: AEDT installation Path C:\Program Files\ANSYS Inc\v252\AnsysEM
PyAEDT INFO: Client application successfully started.
PyAEDT INFO: New AEDT gRPC session session started on port 50051.
PyAEDT INFO: 2025.2 version started with process ID 22700.
PyAEDT WARNING: Service Pack is not detected. PyAEDT is currently connecting in Insecure Mode.
PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Pyt

TypeError: 'NoneType' object does not support item assignment

In [2]:
import math



def winding_points(**kwargs) :

    """
    사용 예시

    winding_params1 = {
        "N" : sim1.N1,
        "N_layer" : sim1.N1_layer,
        "x" : "w1 + 2*N1_space_w",
        "y" : "l1 + 2*N1_space_l",
        "coil_diameter" : "N1_coil_diameter",
        "coil_zgap" : "N1_coil_zgap",
        "coil_layer_x_gap" : "N1_layer_gap",
        "coil_layer_y_gap" : "N1_layer_gap",
        "color" : [255, 50, 50],
        "transparency" : 0,
        "offset" : ["0mm", "-(l1+l2)/2", f"{offset1}mm"]
    }
    points1 = sim1.winding_points(name="winding1", **winding_params1)

    
    """

    """
    예시

    modeler = Transformer_winding(simulation.design)
    winding_point = modeler.winding_points(**kwargs)
    winding = modeler.create_polyline(name=name, points=winding_point,**kwargs)

    """

    raw_N = kwargs.get("N", 10)
    raw_N_layer = kwargs.get("N_layer", 1)

    # N과 N_layer는 Python 로직(range, if)에 사용되므로 정수여야 합니다.
    # 문자열로 들어올 경우, 정수로 변환을 시도하고, 실패하면 디자인 변수로 간주하여 값을 가져옵니다.

    N = int(raw_N)
    N_layer = int(raw_N_layer)


    x = kwargs.get("x", "100mm")
    y = kwargs.get("y", "20mm")
    coil_diameter = kwargs.get("coil_diameter", "10mm")
    coil_zgap = kwargs.get("coil_zgap", "1mm")
    coil_layer_x_gap = kwargs.get("coil_layer_x_gap", "1mm")
    coil_layer_y_gap = kwargs.get("coil_layer_y_gap", "1mm")

    clock_wise = kwargs.get("clock_wise", True)

    terminal = kwargs.get("terminal", True)
    terminal_position = kwargs.get("terminal_position", None)
    if terminal_position is None :
        terminal_length = kwargs.get("terminal_length", "30mm")
        terminal = False
    

    # offset을 x, y, z 각 축에 대한 리스트로 처리합니다.
    offset = kwargs.get("offset", ["0mm", "0mm", "0mm"])
    offset_x, offset_y, offset_z = offset[0], offset[1], offset[2]


    points = []

    
    if clock_wise :
        # --- 레이어 1 좌표 변수 ---
        pos_x1 = f"({x})/2 + ({coil_diameter})/2"
        neg_x1 = f"-({x})/2 - ({coil_diameter})/2"
        pos_y1 = f"({y})/2 + ({coil_diameter})/2"
        neg_y1 = f"-({y})/2 - ({coil_diameter})/2"

        if N_layer == 1 :
            for i in range(N) :
                z_base = f"-({i})*(({coil_diameter}) + ({coil_zgap}))"
                z_start = f"{z_base} + ({offset_z})"
                z_end = f"-({i+1})*(({coil_diameter}) + ({coil_zgap})) + ({offset_z})"

                # 각 좌표에 오프셋 적용
                px1_off = f"{pos_x1} + ({offset_x})"
                nx1_off = f"{neg_x1} + ({offset_x})"
                py1_off = f"{pos_y1} + ({offset_y})"
                ny1_off = f"{neg_y1} + ({offset_y})"

                if terminal and i == 0 :
                    if terminal_position is not None :
                        points.append([f"{terminal_position}", ny1_off, z_start])
                    else :
                        points.append([f"{px1_off} + {terminal_length}", ny1_off, z_start])
                
                points.append([px1_off, ny1_off, z_start])
                points.append([nx1_off, ny1_off, z_start])
                points.append([nx1_off, py1_off, z_start])
                points.append([px1_off, py1_off, z_end])

                if terminal and i == N-1 :
                    if terminal_position is not None :
                        points.append([f"{terminal_position}", py1_off, z_end])
                    else :
                        points.append([f"{px1_off} + {terminal_length}", py1_off, z_end])
                        
        if N_layer == 2 :
            # --- 레이어 2 좌표 변수 ---
            x2 = f"({x})/2 + ({coil_diameter}) + ({coil_layer_x_gap})"
            y2 = f"({y})/2 + ({coil_diameter}) + ({coil_layer_y_gap})"
            pos_x2 = f"({x2}) + ({coil_diameter})/2"
            neg_x2 = f"-({x2}) - ({coil_diameter})/2"
            pos_y2 = f"({y2}) + ({coil_diameter})/2"
            neg_y2 = f"-({y2}) - ({coil_diameter})/2"

            n1 = math.ceil(N / 2) # 첫 번째 레이어의 턴 수

            for i in range(N):
                # --- Z 좌표 계산 ---
                if i < n1: # 레이어 1 (내려가는 중)
                    z_index_start = i
                    z_index_end = i + 1
                else: # 레이어 2 (올라오는 중)
                    z_index_start = n1 - (i - n1)
                    z_index_end = z_index_start - 1
                
                z_base_start = f"-({z_index_start})*(({coil_diameter}) + ({coil_zgap}))"
                z_base_end = f"-({z_index_end})*(({coil_diameter}) + ({coil_zgap}))"
                z_start = f"{z_base_start} + ({offset_z})"
                z_end = f"{z_base_end} + ({offset_z})"

                # 각 좌표에 오프셋 적용
                px1_off = f"{pos_x1} + ({offset_x})"
                nx1_off = f"{neg_x1} + ({offset_x})"
                py1_off = f"{pos_y1} + ({offset_y})"
                ny1_off = f"{neg_y1} + ({offset_y})"
                px2_off = f"{pos_x2} + ({offset_x})"
                nx2_off = f"{neg_x2} + ({offset_x})"
                py2_off = f"{pos_y2} + ({offset_y})"
                ny2_off = f"{neg_y2} + ({offset_y})"


                # --- X, Y 좌표 및 포인트 추가 ---
                if i < n1: # 레이어 1
                    if terminal and i == 0:
                        if terminal_position is not None :
                            points.append([f"{terminal_position}", ny1_off, z_start])
                        else :
                            points.append([f"{px1_off} + {terminal_length}", ny1_off, z_start])
                    points.append([px1_off, ny1_off, z_start])
                    points.append([nx1_off, ny1_off, z_start])
                    points.append([nx1_off, py1_off, z_start])
                    points.append([px1_off, py1_off, z_end])
                elif i == n1: # 레이어 2의 시작 (전이 구간)
                    # 요청하신대로, 올라오는 구간의 첫 포인트는 pos_x1, neg_y2를 사용합니다.
                    points.append([px1_off, ny2_off, z_start]) # 전이 포인트
                    points.append([nx2_off, ny2_off, z_start])
                    points.append([nx2_off, py2_off, z_start])
                    points.append([px2_off, py2_off, z_end])
                else: # 레이어 2의 나머지
                    points.append([px2_off, ny2_off, z_start])
                    points.append([nx2_off, ny2_off, z_start])
                    points.append([nx2_off, py2_off, z_start])
                    points.append([px2_off, py2_off, z_end])

                    if terminal and i == N-1 :
                        if terminal_position is not None :
                            points.append([f"{terminal_position}", py2_off, z_end])
                        else :
                            points.append([f"{px2_off} + {terminal_length}", py2_off, z_end])

                    

    return points




def create_polyline(design, name="winding", points=None, **kwargs):
    """
    폴리라인을 생성합니다.
    
    Parameters:
        name (str): 폴리라인의 이름
        points (list): 폴리라인의 점들
        
    Returns:
        object: 생성된 폴리라인 객체
    """

    x_section_type = kwargs.get("type", "Circle") # Rectangle, Circle
    if x_section_type == "Circle" :
        orientation = kwargs.get("orientation", "Auto")
        xsection_topwidth = kwargs.get("top_width", "0mm")
        xsection_width = kwargs.get("coil_diameter", "5mm")
        xsection_height = kwargs.get("height", "0mm")
    elif x_section_type == "Rectangle" :
        orientation = kwargs.get("orientation", "Auto")
        xsection_topwidth = kwargs.get("top_width", "0mm")
        xsection_width = kwargs.get("width", "5mm")
        xsection_height = kwargs.get("height", "0.5mm")
    elif x_section_type == "Isosceles Trapezoid" :
        orientation = kwargs.get("orientation", "Auto")
        xsection_topwidth = kwargs.get("top_width", "3mm")
        xsection_width = kwargs.get("width", "5mm")
        xsection_height = kwargs.get("height", "0.5mm")
    else :
        raise ValueError(f"Invalid x_section_type: {x_section_type}")
    
    color = kwargs.get("color", None)
    transparency = kwargs.get("transparency", None)
    xsection_num_seg = kwargs.get("Num", 6)
    
    
    if points is None :
        raise ValueError("points is None")

    polyline_obj = design.modeler.create_polyline(
        points=points, name=name, xsection_orient=orientation,
        xsection_type=x_section_type, xsection_width=xsection_width, xsection_height=xsection_height, xsection_num_seg=xsection_num_seg, xsection_topwidth=xsection_topwidth)
    polyline_obj.point_list = points

    if color is not None :
        polyline_obj.color = color
    if transparency is not None :
        polyline_obj.transparency = transparency

    return polyline_obj





In [3]:
points = winding_points()


create_polyline(simulation_runner.maxwell_design, "winding1", points)

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


<frozen importlib._bootstrap>:241: UserWarning: The PyVista module is required to run functionalities of ansys.aedt.core.visualization.advanced.misc.
Install with 

pip install pyvista
INFO:Global:Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


INFO:Global:Materials class has been initialized! Elapsed time: 0m 0sec


In [4]:
test_points

NameError: name 'test_points' is not defined

In [5]:
from ansys.aedt.core.modeler.cad.polylines import PolylineSegment

x = "(100mm)"
y = "(50mm)"
coil_diameter = "(10mm)"
R_coil = "(25mm)"
z_coil_gap = "(5mm)"
coil_gap = f"({coil_diameter}) + ({z_coil_gap})"


x = f"({x} + {coil_diameter}/2)"
y = f"({y} + {coil_diameter}/2)"


test_points = []
seg_type = []

num_seg = 3



N = 1

for i in range(N) :

    if i == 0 :
        test_points.append([f"{x}", f"-{y}", f"-({i}+0.0)*({coil_gap})"])
        test_points.append([f"({x}-{R_coil})", f"-{y}", f"-({i}+0.0)*({coil_gap})"])
        seg_type.append(PolylineSegment(segment_type="Line"))
        test_points.append([f"-({x}-{R_coil})", f"-{y}", f"-({i}+0.5)*({coil_gap})"])
        seg_type.append(PolylineSegment(segment_type="Line"))
    else :
        test_points.append([f"-({x}-{R_coil})", f"-{y}", f"-({i}+0.5)*({coil_gap})"])
        seg_type.append(PolylineSegment(segment_type="Line"))


    seg_type.append(PolylineSegment(segment_type="AngularArc", arc_center=[f"-({x}-{R_coil})", f"-({y}-{R_coil})", f"-({i}+0.5)*({coil_gap})"], arc_angle="-90deg", arc_plane="XY", num_seg=num_seg))


    test_points.append([f"-({x})", f"({y}-{R_coil})", f"-({i}+0.5)*({coil_gap})"])
    seg_type.append(PolylineSegment(segment_type="Line"))


    seg_type.append(PolylineSegment(segment_type="AngularArc", arc_center=[f"-({x}-{R_coil})", f"({y}-{R_coil})", f"-({i}+0.5)*({coil_gap})"], arc_angle="-90deg", arc_plane="XY", num_seg=num_seg))


    test_points.append([f"({x}-{R_coil})", f"({y})", f"-({i}+1.0)*({coil_gap})"])
    seg_type.append(PolylineSegment(segment_type="Line"))


    seg_type.append(PolylineSegment(segment_type="AngularArc", arc_center=[f"({x}-{R_coil})", f"({y}-{R_coil})", f"-({i}+1.0)*({coil_gap})"], arc_angle="-90deg", arc_plane="XY", num_seg=num_seg))


    test_points.append([f"({x})", f"-({y}-{R_coil})", f"-({i}+1.0)*({coil_gap})"])
    seg_type.append(PolylineSegment(segment_type="Line"))


    if i != N-1 :
        seg_type.append(PolylineSegment(segment_type="AngularArc", arc_center=[f"({x}-{R_coil})", f"-({y}-{R_coil})", f"-({i}+1.0)*({coil_gap})"], arc_angle="-90deg", arc_plane="XY", num_seg=num_seg))



# test_points = [
#     [f"{x}", f"-{y}", "0mm"],
#     ["100mm", "20mm", "0mm"],
#     ["71mm", "71mm", "0mm"],
#     ["0mm", "100mm", "0mm"],
# ]
# seg_type = ["Line", "Arc"]

# winding = simulation_runner.maxwell_design.modeler.create_polyline(test_points, segment_type=seg_type, name="PL_line_plus_arc", xsection_type="Circle", xsection_width=coil_diameter, xsection_num_seg=6)

winding = simulation_runner.maxwell_design.modeler.create_polyline(test_points, segment_type=seg_type, name="PL_line_plus_arc")


winding.color = [255, 50, 50]
winding.transparency = 0
# winding = simulation_runner.maxwell_design.modeler.create_polyline(test_points, segment_type=seg_type, name="PL_line_plus_arc")


In [ ]:
import time
from ansys.aedt.core.modeler.cad.polylines import PolylineSegment

# 타이밍 체크 함수
def print_time(message, start, end):
    print(f"{message}: {end - start:.4f} seconds")

# 변수 설정 타이밍
t0 = time.time()

x = "(100mm)"
y = "(50mm)"
coil_diameter = "(10mm)"
R_coil = "(25mm)"
z_coil_gap = "(5mm)"
coil_gap = f"({coil_diameter}) + ({z_coil_gap})"

x = f"({x} + {coil_diameter}/2)"
y = f"({y} + {coil_diameter}/2)"

test_points = []
seg_type = []

num_seg = 3
N = 10

t1 = time.time()
print_time("Variables 정의 및 초기화", t0, t1)

# 루프 타이밍 측정
t2 = time.time()
for i in range(N):

    if i == 0:
        test_points.append([f"{x}", f"-{y}", f"-({i}+0.0)*({coil_gap})"])
        test_points.append([f"({x}-{R_coil})", f"-{y}", f"-({i}+0.0)*({coil_gap})"])
        seg_type.append(PolylineSegment(segment_type="Line"))
        test_points.append([f"-({x}-{R_coil})", f"-{y}", f"-({i}+0.5)*({coil_gap})"])
        seg_type.append(PolylineSegment(segment_type="Line"))
    else:
        test_points.append([f"-({x}-{R_coil})", f"-{y}", f"-({i}+0.5)*({coil_gap})"])
        seg_type.append(PolylineSegment(segment_type="Line"))

    seg_type.append(
        PolylineSegment(
            segment_type="AngularArc",
            arc_center=[f"-({x}-{R_coil})", f"-({y}-{R_coil})", f"-({i}+0.5)*({coil_gap})"],
            arc_angle="-90deg",
            arc_plane="XY",
            num_seg=num_seg,
        )
    )

    test_points.append([f"-({x})", f"({y}-{R_coil})", f"-({i}+0.5)*({coil_gap})"])
    seg_type.append(PolylineSegment(segment_type="Line"))

    seg_type.append(
        PolylineSegment(
            segment_type="AngularArc",
            arc_center=[f"-({x}-{R_coil})", f"({y}-{R_coil})", f"-({i}+0.5)*({coil_gap})"],
            arc_angle="-90deg",
            arc_plane="XY",
            num_seg=num_seg,
        )
    )

    test_points.append([f"({x}-{R_coil})", f"({y})", f"-({i}+1.0)*({coil_gap})"])
    seg_type.append(PolylineSegment(segment_type="Line"))

    seg_type.append(
        PolylineSegment(
            segment_type="AngularArc",
            arc_center=[f"({x}-{R_coil})", f"({y}-{R_coil})", f"-({i}+1.0)*({coil_gap})"],
            arc_angle="-90deg",
            arc_plane="XY",
            num_seg=num_seg,
        )
    )

    test_points.append([f"({x})", f"-({y}-{R_coil})", f"-({i}+1.0)*({coil_gap})"])
    seg_type.append(PolylineSegment(segment_type="Line"))

    if i != N - 1:
        seg_type.append(
            PolylineSegment(
                segment_type="AngularArc",
                arc_center=[f"({x}-{R_coil})", f"-({y}-{R_coil})", f"-({i}+1.0)*({coil_gap})"],
                arc_angle="-90deg",
                arc_plane="XY",
                num_seg=num_seg,
            )
        )
t3 = time.time()
print_time("Polyline points/segment 생성 루프", t2, t3)

# Polyline 생성 시 타이밍 측정
t4 = time.time()


oeditor = simulation_runner.maxwell_design.modeler.oeditor
# 대부분의 AEDT 버전에서 지원
oeditor.SuspendUpdate()


winding = simulation_runner.maxwell_design.modeler.create_polyline(
    test_points, segment_type=seg_type, name="PL_line_plus_arc"
)

oeditor.ResumeUpdate()


t5 = time.time()
print_time("create_polyline 실행", t4, t5)

winding.color = [255, 50, 50]
winding.transparency = 0


Variables 정의 및 초기화: 0.0000 seconds
Polyline points/segment 생성 루프: 0.0000 seconds


GrpcApiError: Failed to execute gRPC AEDT command: SuspendUpdate

In [ ]:
def calculate_error_rate(expected, actual):
    """
    두 값의 오차율(%)을 계산합니다.
    expected: 이론값 또는 기준값
    actual: 측정값 또는 실제값
    return: 오차율 (%)
    """
    try:
        error_rate = abs((actual - expected) / expected) * 100
        return error_rate
    except ZeroDivisionError:
        return float('inf')
    except Exception as e:
        print(f"오차율 계산 중 에러 발생: {e}")
        return None

# 예시 사용법:
expected_value = 9.71
measured_value = 10.14
err = calculate_error_rate(expected_value, measured_value)
print(f"오차율: {err:.2f}%")


오차율: 4.43%


In [11]:
Vp = 750
A = 60e-3 * 160e-3
w = 20e+3 * 2 * 3.141592
N = 5

Vp/A/w/N

0.12433982515870934

In [96]:
import inspect
print(inspect.signature(winding))

TypeError: <ansys.aedt.core.modeler.cad.polylines.Polyline object at 0x000001C730F33C50> is not a callable object

In [40]:
test_points_1 = [[0.4, 0, 0], [-0.4, -0.6, 0], [0.4, 0, 0]]
P6 = simulation_runner.maxwell_design.modeler.create_polyline(
    points=test_points_1,
    segment_type=[
        PolylineSegment(
            segment_type="AngularArc", arc_center=[0, 0, 0], arc_angle="180deg", arc_plane="XY"
        ),
        PolylineSegment(segment_type="Line"),
        PolylineSegment(
            segment_type="AngularArc", arc_center=[0, -0.6, 0], arc_angle="180deg", arc_plane="XY"
        ),
        PolylineSegment(segment_type="Line"),
    ],
)

In [37]:
dir(winding)

winding.end_point

[-50.0, -89.99999999999997, 0.0]

## ParameterManager 라이브러리 사용 예시

하드코딩된 `create_input_parameter` 대신 정형화된 파라미터 관리 라이브러리를 사용하는 방법입니다.


In [ ]:
# ParameterManager 라이브러리 import
from module.parameter_manager import (
    ParameterSpace, 
    SimulationParameterManager,
    SamplingStrategy
)

# 1. 파라미터 공간 정의 (하드코딩 대신)
parameter_space = ParameterSpace()

# 기존 parameter 딕셔너리에서 파라미터 추가
# 예시: 실제 파라미터 이름과 범위는 프로젝트에 맞게 수정 필요
parameter_space.add_parameter("N1", min_value=5, max_value=20, current_value=10, unit="turns")
parameter_space.add_parameter("N1_layer", min_value=1, max_value=3, current_value=2, unit="layers")
parameter_space.add_parameter("w1", min_value=50e-3, max_value=200e-3, current_value=100e-3, unit="m")
parameter_space.add_parameter("l1", min_value=50e-3, max_value=200e-3, current_value=100e-3, unit="m")
# ... 필요한 모든 파라미터 추가

# ParameterManager 초기화
param_manager = SimulationParameterManager(parameter_space)

# 현재 파라미터 값 가져오기 (기존 create_input_parameter와 동일한 형식)
input_parameters = param_manager.get_current_parameters()
print("Input parameters:", input_parameters)


In [ ]:
# Simulation 클래스에 ParameterManager 통합 예시
class SimulationWithPM(Simulation):
    """ParameterManager를 사용하는 Simulation 클래스"""
    
    def __init__(self, parameter_space: ParameterSpace, parameter=None):
        super().__init__(parameter=parameter)
        self.param_manager = SimulationParameterManager(parameter_space)
    
    def create_input_parameter(self):
        """ParameterManager를 사용하여 input_parameter 생성"""
        input_parameter = self.param_manager.get_current_parameters()
        logging.info(f"input_parameter : {input_parameter}")
        logging.info("input_parameter : " + ",".join(str(float(v)) for v in input_parameter.values()))
        return input_parameter
    
    def run_optimization_loop(self, target_outputs, tolerance, n_iterations=5, n_samples=10):
        """
        최적화 루프: 여러 번 FEA 실행 -> regression 학습 -> boundary 조정
        
        Args:
            target_outputs: 목표 출력 (예: {'efficiency': 0.98, 'loss': 80.0})
            tolerance: 허용 오차 (예: {'efficiency': 0.01, 'loss': 10.0})
            n_iterations: 최적화 반복 횟수
            n_samples: 각 반복당 샘플 개수
        """
        for iteration in range(n_iterations):
            logging.info(f"\n=== Iteration {iteration + 1}/{n_iterations} ===")
            
            # 샘플 생성
            samples = self.param_manager.get_next_parameters(
                n_samples=n_samples,
                strategy=SamplingStrategy.LHS
            )
            
            # 각 샘플에 대해 시뮬레이션 실행
            for params in samples:
                self.param_manager.parameter_space.set_current_values(params)
                input_params = self.create_input_parameter()
                self.set_variable(input_params)
                
                # 시뮬레이션 실행 (실제 구현 필요)
                # outputs = self.run_simulation()
                # self.param_manager.record_simulation_result(params, outputs, target_outputs)
            
            # Boundary 조정
            if iteration < n_iterations - 1:
                self.param_manager.optimize_boundaries(
                    target_outputs=target_outputs,
                    tolerance=tolerance,
                    method="regression"  # 또는 "best_performance", "constraint"
                )
        
        return self.param_manager.get_history()

# 사용 예시
# sim_pm = SimulationWithPM(parameter_space)
# history = sim_pm.run_optimization_loop(
#     target_outputs={'efficiency': 0.98},
#     tolerance={'efficiency': 0.01},
#     n_iterations=5
# )


In [ ]:
import math

theta = 90*math.pi/180

[xc, yc, zc] = [0, 0, 0]

[x1, y1, z1] = [5, 0, 0]







